## 歌曲序列建模
by [@寒小阳](http://blog.csdn.net/han_xiaoyang)

### NLP场景下 - One Shot 词编码
我是中国人 => 我 是 中国 人  # 先分词<br>   
我 => [1,0,0,0]  # 再编码 1*500<br>
是 => [0,1,0,0]   1乘以500的向量<br>
...<br>
1*4 vector
one-hot encoding

将word 映射成向量=> vector

distance similarity 

#### 判断周边的环境 - google word2vec 可以在CPU上执行 - 窗口大小
###  Online: C++ 
###  Offline: Gensim -> python 的Word2Vec

![](./word2vec.png)

```python
# word2vec 获得习近平 周边的信息
result = model.most_similar(u"习近平")
for e in result:
    print e[0], e[1]
```
```     
胡锦涛 0.809472680092
江泽民 0.754633367062
李克强 0.739740967751
贾庆林 0.737033963203
曾庆红 0.732847094536
吴邦国 0.726941585541
总书记 0.719057679176
李瑞环 0.716384887695
温家宝 0.711952567101
王岐山 0.703570842743
```

说起来word2vec，其实就是把词映射成一定维度的稠密向量，同时保持住词和词之间的关联性，主要体现在(欧式)距离的远近上。

那么问题来了，word2vec为什么能够学习到这样的结果？

因为我们相信“物以类聚，人以群分” “一个人的层次与他身边最近的一些人是差不多的”

同样的考虑，我们是不是可以认为，一个歌单里的歌曲，相互之间都有一定的关联性呢？就像句子中的词一样。答案是，是的！

咱们来写个程序跑一把。

### 从word2vec到song2vec

我们把歌曲的id序列取出来，类比于分完词后的句子，送到word2vec中去学习一下，看看会有什么效果。

In [12]:
#coding: utf-8
# 当做模型：用于序列建模
import multiprocessing
import gensim # NLP 中的主要model,涵盖word2vec
import sys
from random import shuffle

# 解析歌单，将序列存在plylist
def parse_playlist_get_sequence(in_line, playlist_sequence):  # playlist_sequence is output file
	song_sequence = []
	print "*** in_line is*** ", in_line
	contents = in_line.strip().split("\t") # 空格分割
	#print "contents >>>>> is ",contents
	#print "contents[1:] >>>>> is ",contents[1:]
	# 解析歌单序列
	for song in contents[1:]:
		try:
			song_id, song_name, artist, popularity = song.split(":::")
			song_sequence.append(song_id)
		except:
			print "song format error"
			print song+"\n"
	for i in range(len(song_sequence)): #word2vec 窗口为7
		shuffle(song_sequence)
		playlist_sequence.append(song_sequence)


def train_song2vec(in_file, out_file):
	#所有歌单序列
	playlist_sequence = []
	#遍历所有歌单
	for line in open(in_file):  # line 指的是从in_file 读取的每一 “行” 的文本数据
		#print "line is >>>", line 
		parse_playlist_get_sequence(line, playlist_sequence)
	#使用word2vec训练
	cores = multiprocessing.cpu_count()
	print "using all "+str(cores)+" cores"
	print "Training word2vec model..."
    # min_count = 3，最少的出现次数
    # window = 7 窗口长度为7
    # workers=cores 双核处理
	model = gensim.models.Word2Vec(sentences=playlist_sequence, size=150, min_count=3, window=7, workers=cores)
	print "Saving model..."
	model.save(out_file)

### 整个训练的过程实际是针对某首歌曲，查找到“最近的”歌曲 （向量距离最近的歌曲）

In [13]:
song_sequence_file = "./RawData/popular.playlist"  # 在歌单playlist 里面 取出一部分华语流行的部分
model_file = "./InternalData/song2vec.model"

# 下面是用于训练的过程
%time train_song2vec(song_sequence_file, model_file)

*** in_line is***  【情怀录】泱泱华夏，千古风华##古风,华语,怀旧##392991828##36319	33891487:::礼仪之邦:::HITA:::100.0	31168297:::俯仰河山:::吾恩:::100.0	101085:::为龙:::河图:::100.0	407761300:::踏行:::安九、叶里:::90.0	32432818:::君临天下:::小魂:::90.0	285177:::千年流光:::清莞:::85.0	149875:::重回汉唐:::孙异:::85.0	184408:::笔墨稠:::音频怪物:::100.0	29535956:::声律启蒙:::哈辉:::90.0	28381289:::雅乐寻踪:::i2star:::90.0	185882:::本草纲目:::周杰伦:::100.0	359939:::千年食谱颂:::洛天依:::100.0	31789164:::同衣:::平纱落雁音乐团队:::45.0	33789691:::年事:::冬子:::60.0	29772419:::二十四节气歌:::李秋林:::70.0	28577127:::贺新婚:::小义:::100.0	31587571:::百家姓:::极泷:::40.0	30612309:::李:::李玉刚:::95.0	105863:::中国字画:::极泷:::55.0	33872294:::千本中国通史:::兮曰Crandy:::90.0	459555084:::万神纪（人声本家·正式版）:::肥皂菌丨珉珉的猫咪丨:::100.0	366409:::女娲:::墨明棋妙:::85.0	27946227:::纪年轩辕刺:::ZHUCOOL:::90.0	81635:::哪吒:::EDIQ:::95.0	31421175:::诸子百家:::群星:::95.0	26619450:::春秋争霸:::极泷:::60.0	29747545:::大秦 酬知己:::小魂:::90.0	33314024:::君梦成骸:::吾恩:::90.0	34586733:::千秋月别西楚将:::王胖子:::100.0	31587566:::大风歌:::极泷:::85.0	28535272:::三国物语 序章 乱世长歌:::群星:::90.0	29019518:::宇内三分:::五色石南叶:

*** in_line is***  周杰伦地表最强演唱会2017520南京站曲目##华语##559016045##366	418602085:::英雄:::周杰伦:::100.0	185817:::无双:::周杰伦:::100.0	186021:::她的睫毛:::周杰伦:::100.0	186125:::开不了口:::周杰伦:::100.0	415792916:::床边故事:::周杰伦:::100.0	29822010:::窃爱:::周杰伦:::100.0	186014:::以父之名:::周杰伦:::100.0	29822012:::美人鱼:::周杰伦:::100.0	298317:::屋顶:::温岚:::100.0	399411603:::Try:::派伟俊:::100.0	36024848:::我的时代:::派伟俊:::95.0	185709:::稻香:::周杰伦:::100.0	186016:::晴天:::周杰伦:::100.0	185890:::白色风车:::周杰伦:::100.0	186017:::三年二班:::周杰伦:::100.0	186004:::将军:::周杰伦:::100.0	186116:::爸我回来了:::周杰伦:::100.0	29818117:::鞋子特大号:::周杰伦:::100.0	186046:::半岛铁盒:::周杰伦:::100.0	186158:::印第安老斑鸠:::周杰伦:::100.0	25641372:::大笨钟:::周杰伦:::100.0	186047:::暗号:::周杰伦:::100.0	186156:::伊斯坦堡:::周杰伦:::100.0	186160:::龙卷风:::周杰伦:::100.0	418602086:::土耳其冰淇淋:::周杰伦:::100.0	417247652:::Now You See Me:::周杰伦:::100.0	185694:::给我一首歌的时间:::周杰伦:::100.0	185668:::烟花易冷:::周杰伦:::100.0	186145:::可爱女人:::周杰伦:::100.0	418603077:::告白气球:::周杰伦:::100.0	186053:::爷爷泡的茶:::周杰伦:::100.0	186001:::七里香:::周杰伦:::100.0	186139:::安静:::周杰

*** in_line is***  雾霾诗集##华语,流行,伤感##542417489##3395	33668366:::霾:::洛天依:::60.0	26466808:::PM2.5:::dBu:::25.0	5268960:::粉雾:::唐朝:::25.0	64202:::呼吸困难(Live) - live:::陈奕迅:::65.0	35447858:::走着走着就散了 :::庄心妍:::100.0	22854014:::城市的天空:::牛奶咖啡:::95.0	255899:::空气污染指数:::梁咏琪:::20.0	448116225:::我在阳台上看霾:::胡不一:::20.0	39443861:::吓死宝宝了:::周艳泓:::75.0	28724363:::雾都孤儿:::汪苏泷:::100.0	108590:::回到花开的那天:::林一峰:::20.0	213160:::变脸:::陈琳:::80.0	406000777:::在雾霾天唱歌:::坡上村:::30.0	28254000:::北京爱情:::小柯:::100.0	108098:::北京的冬天:::老狼:::95.0	403710369:::怎么办:::布衣乐队:::95.0	29378734:::冲出霾伏:::零点:::25.0	347690:::送给不懂环保的人:::Beyond:::40.0	27698125:::还我蔚蓝:::反光镜:::100.0	186023:::梯田:::周杰伦:::100.0	409646749:::清夜语:::龙神道:::80.0	255585:::盖亚:::林忆莲:::100.0	438456152:::用爱前行:::Matthew Lien:::25.0	82364:::暖:::方大同:::95.0	150958:::梦开始的地方:::童安格:::90.0	326697:::蓝天白云:::张悬:::95.0	326929:::蓝天:::张惠妹:::95.0	26494738:::地球歌:::黄义达:::80.0	29774171:::新地球:::林俊杰:::100.0	37905454:::爱最大(国语版):::谢霆锋:::55.0

*** in_line is***  你就是我生命中最美丽的比喻句##华语,流行,浪漫##491120233##11255	18

*** in_line is***  [小风收集]21世纪年轻人的音乐##华语,说唱,R&B/Soul##419935738##159	41664255:::未知伴侣:::龚子婕JessieG:::100.0	370776:::二十世纪少年(Unplugged) - unplug:::Ping Pung:::100.0	36668711:::就算我唱遍所有情歌:::简弘亦:::100.0	466370500:::不二臣:::徐秉龙:::100.0	35678973:::非正常励志歌:::齐一:::100.0	277302:::爱:::莫文蔚:::100.0	451126768:::让我留在你身边 (原唱作版):::唐汉霄:::100.0	29803523:::拥抱:::五月天:::100.0	471385043:::暧昧:::薛之谦:::100.0	28234987:::陪你自己:::M.K:::100.0	167876:::有何不可:::许嵩:::100.0	25714352:::空白格:::杨宗纬:::100.0	35476049:::这个年纪:::齐一:::100.0	28773083:::缺口:::庾澄庆:::100.0	432506809:::致姗姗来迟的你 :::阿肆:::100.0	28427066:::放过:::崔天琪:::100.0	399410556:::渐行渐远:::小峰峰:::100.0	461899833:::鸽子:::徐秉龙:::100.0	28481818:::可以了:::陈奕迅:::100.0	410801653:::啷个哩个啷:::鹏泊:::100.0	65528:::淘汰:::陈奕迅:::100.0	30070984:::我唱情歌给你听:::南屋Clean:::95.0	27937072:::玩笑:::刘瑞琦:::100.0	436514312:::成都:::赵雷:::100.0	430043767:::情歌 (cover梁静茹):::简弘亦:::100.0	439915614:::刚好遇见你:::李玉刚:::100.0	439121995:::你一定要幸福 (cover 何洁):::简弘亦:::100.0	41663362:::朋友关系:::龚子婕JessieG:::100.0	399367357:::摊牌:::江潮:::100

*** in_line is***  可惜不能靠得太近，但仍心存感激##华语,粤语,安静##541853248##20647	211707:::距离:::陈慧娴:::90.0	108418:::背对背拥抱:::林俊杰:::100.0	208908:::若你碰到他:::蔡健雅:::100.0	67601:::伤心证明书:::陈奕迅:::100.0	327098:::不像个大人:::张惠妹:::90.0	285095:::我的歌声里:::曲婉婷:::100.0	25657371:::只是会可惜:::梁晓雪:::95.0	209184:::还是会寂寞(Live):::陈绮贞:::100.0	26902925:::爱深过做人:::卫兰:::100.0	30512459:::突然好想你（Cover 五月天）:::许飞:::100.0	223755:::一念之间:::戴佩妮:::95.0	81807:::除此之外:::范逸臣:::95.0	94598:::亲密关系:::郑秀文:::95.0	255864:::迫不得已:::梁咏琪:::80.0	28111253:::想得太远:::容祖儿:::100.0	298090:::眼泪知道:::温岚:::100.0	185908:::黑色毛衣:::周杰伦:::100.0	169257:::Memory:::薛之谦:::100.0	135187:::我有时觉得:::麦浚龙:::90.0	28391394:::敢抱未敢爱:::JW:::90.0	307091:::寻找独角兽:::薛凯琪:::75.0	4875244:::心墙 (插曲):::Bae Fong:::95.0	327096:::人质:::张惠妹:::100.0	299981:::暧昧:::王菲:::100.0	189335:::明了:::张敬轩:::90.0	208901:::抛物线:::蔡健雅:::100.0	28535159:::终点(剧情版):::关心妍:::95.0	4877385:::Piano:::范逸臣:::95.0	190545:::真情流露:::张学友:::95.0	29744130:::点到为止:::江映蓉:::100.0	175977:::被遗忘的:::杨宗纬:::100.0	25730751:::歌和老街:::王祖蓝:::90.0	273405:::今天终于

*** in_line is***  致高考|你是自己不灭的火焰##学习,校园,华语##593496993##43262	423015580:::热勇:::栗先达:::100.0	36496726:::光:::刘惜君:::100.0	436346799:::飞:::谭维维:::100.0	34928388:::光:::姚贝娜:::100.0	443291080:::别怕:::阡陌:::85.0	5250695:::Run:::朱婧汐:::90.0	286096:::少年:::任月丽:::90.0	306648:::起点:::徐佳莹:::95.0	146723:::曙光:::石康军:::85.0	33337096:::野子:::云の泣:::100.0	101390:::梦想:::胡夏:::95.0	286092:::年华:::任月丽:::90.0	254029:::远行:::李慧珍:::90.0	29710870:::重生:::王博文:::95.0	28665060:::别怕:::徐佳莹:::90.0	26348314:::标本:::周笔畅:::80.0	156154:::青春:::王梵瑞:::100.0	41500601:::平凡曲:::大宽:::75.0	368725:::去寻找:::牛奶咖啡:::95.0	409102911:::Home:::王诗安:::100.0	108406:::加油！:::MC Hotdog:::100.0	452997361:::光芒:::程璧:::100.0	33346401:::换向:::樊凡:::85.0	439122777:::回合:::金玟岐:::100.0	26609640:::楼道:::刘美麟:::90.0	327208:::寓言:::张韶涵:::100.0	29567189:::理想:::赵雷:::100.0	32648345:::星空:::五月天:::100.0	190381:::水手:::郑智化:::100.0	28949013:::歌路:::刘瑞琦:::100.0	287057:::逆光:::孙燕姿:::100.0	26609639:::胡说:::刘美麟:::75.0	185916:::逆鳞:::周杰伦:::100.0	455632125:::借我（原唱：谢春花（知非））:::归于藏:::85.0	35447132

KeyboardInterrupt: 

模型已经训练完了，咱们来试一把预测，看看效果

这个预测的过程，实际上就是对某首歌曲，查找“最近”的歌曲（向量距离最近的歌曲）

In [2]:
import cPickle as pickle  # 歌曲Id到歌曲的映射pickle
song_dic = pickle.load(open("popular_song.pkl","rb"))
model_str = "./song2vec.model"
model = gensim.models.Word2Vec.load(model_str)

In [3]:
for song in song_dic.keys()[:10]:
    print song, song_dic[song]

287140 梦不落	孙燕姿
445845011 狂想.Rhapsody	冯建宇
110557 灰色空间	罗志祥
10308003 偏偏喜欢你	陈百强
28029940 拥抱的理由	尹熙水
28029946 三个人的错	王菀之
28029947 拥抱的理由	李泰
27591219 拍错拖	卫兰
28029949 我是你的谁	张含韵
31134863 没有用	徐誉滕


In [8]:
song_id_list = song_dic.keys()[1000:1500:50]
for song_id in song_id_list:
    result_song_list = model.most_similar(song_id)

    print song_id, song_dic[song_id]
    print "\n相似歌曲 和 相似度 分别为:"
    for song in result_song_list:
        print "\t", song_dic[song[0]], song[1]
    print "\n"

440357032 七里香	李蚊香

相似歌曲 和 相似度 分别为:
	东风志 钢琴竹笛合奏	昼夜 0.995016634464
	少年熬至老	少恭 0.994233191013
	【钢琴】forever love	昼夜 0.98337417841
	并骑(无念白版)	群星 0.976293027401
	造花の距離感（Cover：miku）	李蚊香 0.966003715992
	过我的生活	伦桑 0.950182139874
	3 7 20 1（Cover：曹格）	萧忆情Alex 0.943133831024
	爱如潮水（Cover 张信哲）	柏凝 0.915985286236
	倾君	白止 0.897894084454
	一纸流年	伦桑 0.872422993183


27906826 你有本事抢男人	雪姨

相似歌曲 和 相似度 分别为:
	风云之歌	周思贤 0.901113092899
	普通disco	葛平 0.877337396145
	循环	周天然 0.842635333538
	恋·koigokoro (二胡版)	永安二胡 0.838205397129
	送别	王俊雄 0.829163253307
	入站歌	葛平 0.82365167141
	我是谁	周华健 0.820580601692
	蓝皮鼠和大脸猫	章中锦 0.810702621937
	平凡之路	葛平 0.790481626987
	Rags To Rings	Mark Petrie 0.782146155834


28048161 高地	汪峰

相似歌曲 和 相似度 分别为:
	夜已变得骚了	刘美君 0.99225628376
	垂涎	容祖儿 0.98558139801
	演歌	陈珊妮 0.763966083527
	吸你	蓝奕邦 0.696066439152
	馋	杨宗纬 0.687027215958
	莫呼洛迦(粤)	辛晓琪 0.667171180248
	你骚你骚	花粥 0.65488743782
	叮当	樊竹青 0.611336767673
	一个人生活	刘洪喆 0.600493013859
	深夜港湾	关淑怡 0.583668828011


29482465 I'll be the one	HΛL

相似歌曲 和 相似度 分别为:
	Jai Ho	Alaa Wardi 0

### 进一步思考

所以我们用word2vec学会了哪些歌曲和哪些歌曲最接近。

我们来思考一些很现实同时又很难解决的问题。比如：
#### 1）冷启动问题
我们经常会遇到冷启动的问题，比如没有任何信息的歌曲，我们如何对它做推荐呢？
* 如果是歌手发行的新歌曲，我们怎么进行推荐呢？
* 如果我听完(并收藏)了一首很冷门的歌，怎么进行推荐呢？

我们知道新歌(或者小众的歌)是非常难和其他的歌关联上的，我们有的信息太少了(很少有用户在它上面发生行为)。

1.1 一种解决办法当然是推荐热门的歌曲，但是其实没从个人兴趣出发，我们知道这并不是最好的办法，并没有太大的卵用。

1.2 我们把问题的粒度放粗一点，用同样的思路，比如一个可考虑的解决方案是，我们把**歌曲**的粒度上升到对应的**歌手**，把刚才的song_list替换成artist_list，重新用word2vec建模，这样我们可以得到和一个歌手最相关(接近)的歌手，再推荐这个歌手最热门的歌曲，相对1.1的方法针对性强一些。

将商品 上升到=> 品类<br>
品类list => 送到word2vec里面去学习<br>
[上衣，上衣，上衣，牛仔裤，牛仔裤,连衣裙...]<br>

#### 2）用户兴趣预测问题
我们刚才完成的功能，类似酷狗音乐和网易音乐里针对一首歌的**“相似音乐”**，那么问题又来了，如果我们现在要对一个user用这套song2vec的方式推荐，我们怎么做呢？

* 每个人的兴趣都是有时效性的，这意味着说，3年前我喜欢王菲的歌，去年我喜欢五月天的歌，而今年我可能就改摇滚路线，喜欢汪峰的歌了。
* 每一首歌的热度也是不一样的，有一些热门的歌，如果用户能喜欢，当然是首选

那么，我们来做一个粗暴一点点的处理，把这2个维度拉进来，一起来针对一个用户做推荐。

**把每个用户喜欢(收藏)过的歌，沿着时间轴排好，同时由近到远给不同的衰减因子(比如最近一首歌是1，前一首是0.98，再前一首是0.98^2，以此类推...)，同时我们针对不同的歌曲热度，给定不同的推荐因子(比如热度100的是1，热度80的是0.9...)，每一首歌都可以拿回一个song2vec的推荐列表和对应的相似度，对相似度以时间衰减因子和热度权重进行加权，最后的结果排序后，展示给用户。